In [18]:
import random

In [19]:
def initialize(filename):
    with open(filename, "r") as f:
        num_users, num_resources = [int(x) for x in f.readline().split()]
        cost = [[int(x) for x in f.readline().split()] for _ in range(num_users)]
        facility_cost = [int(x) for x in f.readline().split()]
    return facility_cost, cost

In [20]:
def is_feasible(solution):
    return any(solution)

In [21]:
class Individual:
    def __init__(self, facility_cost, cost):
        num_resources = len(facility_cost)
        self.code = [random.random() < 0.5 for _ in range(num_resources)]
        self.fitness = self.calc_fitness(facility_cost, cost)

    def __lt__(self, other):
        return self.fitness < other.fitness
    
    def calc_fitness(self, facility_cost, cost):
        if not is_feasible(self.code):
            return float('-inf') # moze suma svih cost i facility_cost

        value = 0
        for i, s in enumerate(self.code):
            if s:
                value += facility_cost[i]

        for i in range(len(cost)):
            min_cost = float('inf')
            for j in range(len(cost[i])):
                if self.code[j] and cost[i][j] < min_cost:
                    min_cost = cost[i][j]
            value += min_cost

        return -value

In [22]:
def selection(population, tournament_size):
    chosen = random.sample(population, tournament_size)
    return max(chosen)

In [23]:
def crossover(parent1, parent2, child1, child2):
    random_pos = random.randrange(0, len(parent1.code))
    
    child1.code[:random_pos] = parent1.code[:random_pos]
    child1.code[random_pos:] = parent2.code[random_pos:]
    
    child2.code[:random_pos] = parent2.code[:random_pos]
    child2.code[random_pos:] = parent1.code[random_pos:]

In [24]:
def mutation(individual, mutation_prob):
    for i in range(len(individual.code)):
        if random.random() < mutation_prob:
            individual.code[i] = not individual.code[i]

In [28]:
def ga(cost, facility_cost, population_size, num_generations, tournament_size, elitism_size, mutation_prob):
    population = [Individual(facility_cost, cost) for _ in range(population_size)]
    new_population = population.copy()
    
    for i in range(num_generations):
        population.sort(reverse=True)
        new_population[:elitism_size] = population[:elitism_size]
        for j in range(elitism_size, population_size, 2):
            parent1 = selection(population, tournament_size)
            parent2 = selection(population, tournament_size)
            
            crossover(parent1, parent2, child1=new_population[j], child2=new_population[j+1])

            mutation(new_population[j], mutation_prob)
            mutation(new_population[j+1], mutation_prob)
            
            new_population[j].fitness = new_population[j].calc_fitness(facility_cost, cost)
            new_population[j+1].fitness = new_population[j+1].calc_fitness(facility_cost, cost)
        
        population = new_population.copy()
    return max(population)

In [29]:
facility_cost, cost = initialize("ga.txt")

In [38]:
best_individual = ga(
    cost=cost,
    facility_cost=facility_cost,
    population_size=100,
    num_generations=10,
    tournament_size=7,
    elitism_size=10,
    mutation_prob=0.05,
  )

In [39]:
-best_individual.fitness

34